In [15]:
import pandas as pd
import numpy as np
import time
import matplotlib.pylab  as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, GridSearchCV,  ParameterGrid
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif, SelectKBest
from sklearn.dummy import DummyClassifier
from sklearn.utils.multiclass import unique_labels


In [16]:
features = pd.read_csv('../data/features_plus_descriptions.csv')
features.set_index('Feature Type and Number', inplace=True)
features.drop(['S5', 'D21'], axis=0, inplace=True)

In [17]:
df = pd.read_csv('../data/multiclass_target_raw_data.csv')
X = df.loc[:,features.index]
y = df['Best Heuristic']

In [18]:
types = dict.fromkeys(features.index, 'stdsc')
mm_feats = ['S1','S3', 'S4', 'S6', 'S8','S11', 'S12', 'D3', 'D39']

for feat in mm_feats:
    types[feat] = 'minmax'

minmax_feats = []
std_feats = []

for feat in types.keys():
    if types[feat] == 'stdsc':
        std_feats.append(feat)
    elif types[feat] == 'minmax':
        minmax_feats.append(feat)

print('MinMax Scaler Features: ', minmax_feats)
print('Standard Scaler Features: ', std_feats)

preprocessor = ColumnTransformer(
    transformers=[
        ('mm_scaler', MinMaxScaler(), minmax_feats),
        ('std_scaler', StandardScaler(), std_feats)])

MinMax Scaler Features:  ['S1', 'S3', 'S4', 'S6', 'S8', 'S11', 'S12', 'D3', 'D39']
Standard Scaler Features:  ['S2', 'S7', 'S9', 'S10', 'S13', 'S14', 'D1', 'D2', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38']


In [19]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['figure.figsize'] = (8,8)
    plt.rcParams['font.size'] = 8
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
  #  classes = np.array(classes)
  #  classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [20]:
def ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid):
    """
    This function splits the data into other and test sets (80-20 split) and
    then applies KFold with 4 folds to other set. 
    """
    test_scores = []
    best_models = []
    ys = []
    for i in range(1, 6):
        random_state = 431*i
        X_other, X_test, y_other, y_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)
        kf = KFold(n_splits=4, shuffle=True, random_state=random_state)
        clf = Pipeline([
        ('preprocessor', preprocessor), 
        ('clf', ML_algo)
        ])
        grid = GridSearchCV(clf, param_grid, scoring='f1_macro',cv=kf,  return_train_score=True, verbose=4, n_jobs=-1)  
        grid.fit(X_other, y_other)
        y_pred = grid.predict(X_test)
        prec = precision_score(y_test, y_pred, average="macro")
        rec = recall_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")
        best_models.append(grid.best_params_)
        print("best params", grid.best_params_, "precision score", prec, "recall score", rec, "f1 score", f1)
        test_scores.append((prec, rec, f1))
        ys.append((y_test, y_pred, [0,1,2,3,4,5]))
    return test_scores, best_models, ys


In [23]:
ML_algo = DummyClassifier(strategy="uniform")
param_grid = { 'clf__random_state': [431, 431*2, 431*3, 431*4, 431*5]}

scores_dummy, models_dummy, ys_dummy = ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid)

for m in models_dummy:
    print(m)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


best params {'clf__random_state': 1293} precision score 0.1548702636583802 recall score 0.16324590340469733 f1 score 0.1459648037024512
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished


best params {'clf__random_state': 1724} precision score 0.16238449353011247 recall score 0.1649415441764084 f1 score 0.14608595351644268
Fitting 4 folds for each of 5 candidates, totalling 20 fits


...

best params {'clf__random_state': 1293} precision score 0.15533817269893752 recall score 0.15888335508849546 f1 score 0.1423048644170937
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.2s finished


best params {'clf__random_state': 431} precision score 0.15733727865373254 recall score 0.1598297804797557 f1 score 0.14390476436659697
Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.1s finished


best params {'clf__random_state': 2155} precision score 0.17386842729916308 recall score 0.16719028036074576 f1 score 0.1559323891856043
{'clf__random_state': 1293}
{'clf__random_state': 1724}
{'clf__random_state': 1293}
{'clf__random_state': 431}
{'clf__random_state': 2155}


In [ ]:
param_grid = { 'clf__max_depth': [15, 20, 25, 30], 
               'clf__max_features':[10, 15, 20, 25, 30] }

ML_algo = RandomForestClassifier()
pg = ParameterGrid(param_grid)
scores_orig_RF, models_orig_RF, ys_orig_RF = ML_pipeline_KFold_orig(X, y, preprocessor, ML_algo, param_grid)

for p, r, f in scores_orig_RF:
    print("precision: {}      , recall: {}, f1: {}".format(p,r, f))
    
for m in models_orig_RF: 
    print(m)

print(type(ys_orig_RF[0][0]))
yt, yp, classes = ys_orig_RF[0][0], ys_orig_RF[1][0], ys_orig_RF[2]
cl = [[0], [1], [2], [3], [4], [5]]
plot_confusion_matrix(yt, yp, cl, normalize=True)